In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import config
import json as json
from config import gmaps_key

Store Part I results into DataFrame
Load the csv exported in Part I to a DataFrame

In [ ]:
#os.getcwd()
cities_vac_df = pd.read_csv('../output_data/cities.csv')
cities_vac_df.dropna(inplace = True) 
cities_vac_df

Humidity Heatmap
Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=gmaps_key)
locations = cities_vac_df[["Lat", "Lon"]].astype(float)
humidity = cities_vac_df["Humidity"].astype(float)

fig = gmaps.figure()

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

# Shoe map
fig

Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [ ]:
#This is kind of bogus because data are for one timepoint only... Could be a sunny day in Bergen, it happens.

cities_pref_df = cities_vac_df
cities_pref_df.drop(cities_pref_df[cities_pref_df['Feels_like'] > 30].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Feels_like'] < -10].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Humidity'] > 80].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Country'] == "US"].index, inplace = True)
cities_pref_df



Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [ ]:
hotel_df = cities_pref_df
hotel_df["Hotel"] = ""

target_radius = 5000
target_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "radius": target_radius,
    "type": target_type,
    "key": gmaps_key
}

#(print log commented out because annoying)
for index, row in hotel_df.iterrows():
    # get lat, lng from df; remaining parameters from paramters dictionary
    lat = row["Lat"]
    lon = row["Lon"]
    params["location"] = f"{lat},{lon}"

    response = requests.get(base_url, params=params).json()
    results = response['results']    
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel"] = results[0]['name']
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel"] = "missing value"
    #print("------------")

hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [ ]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [ ]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig